In [ ]:
import nltk
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
import pickle

train_text = state_union.raw("2005-GWBush.txt")
sample_text = state_union.raw("2006-GWBush.txt")

custom_sent_tokenizer = PunktSentenceTokenizer(train_text)

tokenized = custom_sent_tokenizer.tokenize(sample_text)

def process_content():
    try:
        for i in tokenized[5:]:
            words = nltk.word_tokenize(i)
            tagged = nltk.pos_tag(words)
            namedEnt = nltk.ne_chunk(tagged, binary=True)
            namedEnt.draw()
    except Exception as e:
        print(str(e))

def saveClassifier():
    save_class = open("classifier.pickle", "wb")
    pickle.dump(custom_sent_tokenizer, save_class)
    save_class.close()
    

#saveClassifier()
#process_content()

In [ ]:
import os
import nltk
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import state_union

def findName(line):
    
    train_text = state_union.raw("2005-GWBush.txt")
    
    custom_sent_tokenizer = PunktSentenceTokenizer(train_text)
    
    #text = inputFile.read()
    
    # tokenizing the input file (the main file)
    tokenized = custom_sent_tokenizer.tokenize(line)[0]
    #print(tokenized)
    
    # finding the names
    try:
        words = nltk.word_tokenize(tokenized)
        tagged = nltk.pos_tag(words)
        namedEnt = nltk.ne_chunk(tagged, binary=True)
        #print(namedEnt)
        #print("Broken down: ")
        # determining if there's a name in the title
        # that's such a broad ass descpritor (NNP)
        for ent in namedEnt:
            for entType in ent:
                print(entType)
                if entType[1] == "NNP":
                    print("Found:")
                    print(line)
                    print(entType[0])
                    print("\n\n")
    except:
        print("Error with tokenizing")
    
    
    
    
print("Starting...")
    
inputFile = open("MainList-Copy1.txt", "r")

inputFile.seek(0, os.SEEK_SET)

line = inputFile.readline()

print("Finding name...")
while line:

    #print(line)

    if findName(line):
        # if a name is found in the title, write it out to the new file
        titleLine = line
        urlLine = inputFile.readline()
    else:
        # otherwise, go the next title
        line = inputFile.readline()
    
    line = inputFile.readline()

inputFile.close()

print("Done!")

In [ ]:
import os
import spacy


def findName(line):
    nlp = spacy.load("en_core_web_sm")
    #doc = nlp("Apple is looking at buying U.K. startup for $1 billion")
    doc = nlp(line)
    
    for ent in doc.ents:
        #print(ent.text, ent.label_)
        if ent.label_ == "PERSON":
            if ent.lemma_:
                print(ent.lemma_)
            print("\t\tPeRsOn")
        elif ent.label_ == "ORG" or ent.label_ == "WORK_OF_ART":
            if ent.lemma_:
                print(ent.lemma_)
            print("\t\toRg Or WoRk Of ArT")
    
        
    print('\n')


print("\t\t\tStarting...")
    
inputFile = open("MainList-Copy1.txt", "r")

inputFile.seek(0, os.SEEK_SET)

line = inputFile.readline()

print("\t\tFinding name...")
while line:

    print('\t', line)

    findName(line)
    line = inputFile.readline()
    line = inputFile.readline()

inputFile.close()

print("\t\t\tDone!")

In [ ]:
import os
import nltk
from nltk.tag.stanford import StanfordNERTagger as NERTagger

def findName(line):
    st = NERTagger('../poli_stanford_ner/stanford_ner/english.all.3class.distsim.crf.ser.gz', '../poli_stanford_ner/stanford_ner/stanford-ner-4.2.0.jar')
    
    pos = 0
    savedPos = -1
    multi_name = {}
    ret_names = []
    
    # classifying if there are names in the sentence
    for sent in nltk.sent_tokenize(line):
        tokens = nltk.tokenize.word_tokenize(sent)
        tags = st.tag(tokens)
        for tag in tags:
            if tag[1]=='PERSON': 
                print(tag)
                multi_name[pos] = tag
            pos += 1
    # where it starts to see if there's first, middle, and last names
    keys = isConsecutive(multi_name)
    if keys:
        #print("Multi name!")
        for keySet in keys:
            tmp = None
            for key in keySet:
                if tmp is None:
                    tmp = multi_name[key][0]
                else:
                    tmp += "_" + multi_name[key][0]
            #print("\t\t", tmp)
            ret_names.append(tmp)
    else:
        tmp = None
        for posInLine in multi_name:
            # if this is the first time through
            if savedPos == -1:
                savedPos = posInLine
            if savedPos+1 != posInLine:
                tmp = multi_name[savedPos][0]
                ret_names.append(tmp)
            savedPos = posInLine
    print(ret_names)
    return ret_names
    

'''
Checks if the given dictionary's keys are in (at least partial) sequential order
Must use numeric keys for this 
'''
def isConsecutive(d):
    if len(d) <= 1:
        return
    
    ret_keys = []
    curr_keys = []
    
    keys = list(d.keys())
    #print(keys, len(keys))
    for i in range(len(keys)):
        try:
            # if the keys are sequential add them to the list
            if keys[i+1] - keys[i] == 1:
                #print("Coolio")
                curr_keys.append(keys[i])
                curr_keys.append(keys[i+1])
            else:
                # if they're not, check if there's anything in the current list
                if curr_keys:
                    #print("\t\tBALLIN")
                    # if there is, remove any duplicates, sort it, 
                    # add to return keys, and clear the current list
                    tmp = list(set(curr_keys))
                    tmp.sort()
                    ret_keys.append(tmp)
                    curr_keys.clear()
        except:
            if i == len(keys):
                if curr_keys:
                    #print("\t\t\tBALLING pt2")
                    # if there is, remove any duplicates, sort it, 
                    # add to return keys, and clear the current list
                    tmp = list(set(curr_keys))
                    tmp.sort()
                    ret_keys.append(tmp)
                    curr_keys.clear()
            else:
                print("\t\t\t\t\tOUT OF BOUNDS")
                
            if i == len(keys)-1:
                #print("\tAt boundary")
                tmp = list(set(curr_keys))
                tmp.sort()
                #print(tmp)
                ret_keys.append(tmp)
    
    try:
        if ret_keys[0]:
            print(ret_keys)
            return(ret_keys)
        else:
            #print('\t', keys)
            for key in keys:
                tmpList = []
                #print('\t\t', key)
                tmpList.append(key)
                ret_keys.append(tmpList)
            if not ret_keys[0]:
                del ret_keys[0]
            print(ret_keys)
            return ret_keys
    except:
        pass
    
    
    
    
    
print("\t\t\tStarting...")
    
inputFile = open("MainList-Copy1.txt", "r")

inputFile.seek(0, os.SEEK_SET)

line = inputFile.readline()

print("\t\tFinding names...\n")
while line:

    print('\t', line)

    names = findName(line)
    print(names)
    line = inputFile.readline()
    line = inputFile.readline()
    
    print("----------------------------------------------------------------\n\n")
    
inputFile.close()

print("\t\t\tDone!")